In [1]:
#### GLCM in SEM 8 - Healthy
import numpy as np 
import cv2 
import os
import re

# -------------------- Utility functions ------------------------
def normalize_label(str_):
    str_ = str_.replace(" ", "")
    str_ = str_.translate(str_.maketrans("","", "()"))
    str_ = str_.split("_")
    return ''.join(str_[:2])

def normalize_desc(folder, sub_folder):
    text = folder + " - " + sub_folder 
    text = re.sub(r'\d+', '', text)
    text = text.replace(".", "")
    text = text.strip()
    return text

def print_progress(val, val_len, folder, sub_folder, filename, bar_size=10):
    progr = "#"*round((val)*bar_size/val_len) + " "*round((val_len - (val))*bar_size/val_len)
    if val == 0:
        print("", end = "\n")
    else:
        print("[%s] folder : %s/%s/ ----> file : %s" % (progr, folder, sub_folder, filename), end="\r")
        

# -------------------- Load Dataset ------------------------
 
dataset_dir = "D:/Aishwarya/College/Sem8/Major Project 2/GLCM Healthy" 

imgs = [] #list image matrix 
labels = []
descs = []
for folder in os.listdir(dataset_dir):
    for sub_folder in os.listdir(os.path.join(dataset_dir, folder)):
        sub_folder_files = os.listdir(os.path.join(dataset_dir, folder, sub_folder))
        len_sub_folder = len(sub_folder_files) - 1
        for i, filename in enumerate(sub_folder_files):
            img = cv2.imread(os.path.join(dataset_dir, folder, sub_folder, filename))
            
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            
            h, w = gray.shape
            ymin, ymax, xmin, xmax = h//3, h*2//3, w//3, w*2//3
            crop = gray[ymin:ymax, xmin:xmax]
            
            resize = cv2.resize(crop, (0,0), fx=0.5, fy=0.5)
            
            imgs.append(resize)
            labels.append(normalize_label(os.path.splitext(filename)[0]))
            descs.append(normalize_desc(folder, sub_folder))
            
            print_progress(i, len_sub_folder, folder, sub_folder, filename)

In [2]:
from skimage.feature import greycomatrix, greycoprops

# ----------------- calculate greycomatrix() & greycoprops() for angle 0, 45, 90, 135 ----------------------------------
def calc_glcm_all_agls(img, label, props, dists=[5], agls=[0, np.pi/4, np.pi/2, 3*np.pi/4], lvl=256, sym=True, norm=True):
    
    glcm = greycomatrix(img, 
                        distances=dists, 
                        angles=agls, 
                        levels=lvl,
                        symmetric=sym, 
                        normed=norm)
    feature = []
    glcm_props = [propery for name in props for propery in greycoprops(glcm, name)[0]]
    for item in glcm_props:
            feature.append(item)
    feature.append(label) 
    
    return feature


# ----------------- call calc_glcm_all_agls() for all properties ----------------------------------
properties = ['dissimilarity', 'correlation', 'homogeneity', 'contrast', 'ASM', 'energy']

glcm_all_agls = []
for img, label in zip(imgs, labels): 
    glcm_all_agls.append(
            calc_glcm_all_agls(img, 
                                label, 
                                props=properties)
                            )
 
columns = []
angles = ['0', '45', '90','135']
for name in properties :
    for ang in angles:
        columns.append(name + "_" + ang)
        
columns.append("label")

In [3]:
import pandas as pd 

# Create the pandas DataFrame for GLCM features data
glcm_df = pd.DataFrame(glcm_all_agls, columns = columns)

glcm_df.head(15)

,dissimilarity_0,dissimilarity_45,dissimilarity_90,dissimilarity_135,correlation_0,correlation_45,correlation_90,correlation_135,homogeneity_0,homogeneity_45,...,contrast_135,ASM_0,ASM_45,ASM_90,ASM_135,energy_0,energy_45,energy_90,energy_135,label
0,20.682754,21.129501,21.028958,19.935596,0.091815,0.073319,0.055293,0.151102,0.049701,0.046735,...,648.615651,0.000557,0.000593,0.000562,0.000588,0.023600,0.024342,0.023714,0.024243,SegImage1
1,37.131274,30.035319,26.560489,34.325485,0.060983,0.321365,0.467675,0.126658,0.029487,0.034501,...,2013.835180,0.000396,0.000427,0.000409,0.000426,0.019907,0.020671,0.020232,0.020631,SegImage10
2,15.940154,19.250693,15.892535,12.722992,0.470766,0.272267,0.497191,0.649037,0.071367,0.066940,...,333.376731,0.000634,0.000603,0.000626,0.000701,0.025175,0.024553,0.025018,0.026479,SegImage100
3,14.153153,15.266620,16.029601,16.172438,0.248953,0.151943,0.034582,0.048938,0.084727,0.078121,...,454.756925,0.000782,0.000748,0.000767,0.000778,0.027968,0.027344,0.027697,0.027899,SegImage101
4,12.540541,12.501385,12.361004,13.637812,0.286363,0.283950,0.245652,0.138384,0.076990,0.077559,...,302.549169,0.000866,0.000867,0.000838,0.000863,0.029436,0.029446,0.028947,0.029381,SegImage102
5,20.963320,20.119114,17.129987,17.707756,0.024669,0.022311,0.264681,0.223251,0.056421,0.070217,...,537.367036,0.000601,0.000632,0.000625,0.000661,0.024516,0.025132,0.025010,0.025712,SegImage103
6,13.248391,12.907202,13.683398,13.972992,0.160928,0.169162,0.058715,0.068427,0.078695,0.078729,...,314.245845,0.000843,0.000877,0.000839,0.000858,0.029036,0.029609,0.028968,0.029287,SegImage104
7,20.198842,19.990305,18.747748,19.191136,0.099352,0.136079,0.213707,0.162730,0.059358,0.052947,...,649.682825,0.000619,0.000678,0.000651,0.000657,0.024873,0.026032,0.025510,0.025628,SegImage105
8,21.587516,19.351801,18.272844,19.970222,0.280206,0.382460,0.509752,0.398011,0.051790,0.062631,...,647.840028,0.000495,0.000538,0.000524,0.000525,0.022254,0.023197,0.022896,0.022921,SegImage106
9,20.401544,18.226454,15.564994,22.022161,0.481514,0.538436,0.684198,0.414065,0.059901,0.064424,...,796.768698,0.000551,0.000569,0.000553,0.000558,0.023472,0.023859,0.023516,0.023617,SegImage107


In [4]:
glcm_df.to_csv("D:/Aishwarya/College/Sem8/Major Project 2/GLCM_Healthy.csv")

In [5]:
# GLCM Sem 8 - Unhealthy
dataset_dir = "D:/Aishwarya/College/Sem8/Major Project 2/GLCM Unhealthy" 

imgs = [] #list image matrix 
labels = []
descs = []
for folder in os.listdir(dataset_dir):
    for sub_folder in os.listdir(os.path.join(dataset_dir, folder)):
        sub_folder_files = os.listdir(os.path.join(dataset_dir, folder, sub_folder))
        len_sub_folder = len(sub_folder_files) - 1
        for i, filename in enumerate(sub_folder_files):
            img = cv2.imread(os.path.join(dataset_dir, folder, sub_folder, filename))
            
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            
            h, w = gray.shape
            ymin, ymax, xmin, xmax = h//3, h*2//3, w//3, w*2//3
            crop = gray[ymin:ymax, xmin:xmax]
            
            resize = cv2.resize(crop, (0,0), fx=0.5, fy=0.5)
            
            imgs.append(resize)
            labels.append(normalize_label(os.path.splitext(filename)[0]))
            descs.append(normalize_desc(folder, sub_folder))
            
            print_progress(i, len_sub_folder, folder, sub_folder, filename)

In [6]:
from skimage.feature import greycomatrix, greycoprops

# ----------------- calculate greycomatrix() & greycoprops() for angle 0, 45, 90, 135 ----------------------------------
def calc_glcm_all_agls(img, label, props, dists=[5], agls=[0, np.pi/4, np.pi/2, 3*np.pi/4], lvl=256, sym=True, norm=True):
    
    glcm = greycomatrix(img, 
                        distances=dists, 
                        angles=agls, 
                        levels=lvl,
                        symmetric=sym, 
                        normed=norm)
    feature = []
    glcm_props = [propery for name in props for propery in greycoprops(glcm, name)[0]]
    for item in glcm_props:
            feature.append(item)
    feature.append(label) 
    
    return feature


# ----------------- call calc_glcm_all_agls() for all properties ----------------------------------
properties = ['dissimilarity', 'correlation', 'homogeneity', 'contrast', 'ASM', 'energy']

glcm_all_agls = []
for img, label in zip(imgs, labels): 
    glcm_all_agls.append(
            calc_glcm_all_agls(img, 
                                label, 
                                props=properties)
                            )
 
columns = []
angles = ['0', '45', '90','135']
for name in properties :
    for ang in angles:
        columns.append(name + "_" + ang)
        
columns.append("label")

# Create the pandas DataFrame for GLCM features data
glcm_df = pd.DataFrame(glcm_all_agls, 
                      columns = columns)

glcm_df.head(15)

glcm_df.to_csv("D:/Aishwarya/College/Sem8/Major Project/GLCM_NewUnhealthy.csv")